In [1]:
%load_ext autoreload
%autoreload 2

import torch

from lib.model.progen.init_model import init_model
from lib.model.train import train
from lib.model.progen.init_tokenizer import init_tokenizer
from lib.data.datasets.GB1 import get_GB1_dataset

TEST_SPLIT = 0.1
BATCH_SIZE = 1
LEARNING_RATE = 1e-3
N_EPOCHS = 1

s:\Documents\master\ProGen\progen\progen2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Tokenizer

In [3]:
tokenizer = init_tokenizer()
tokenize = lambda sequence: torch.tensor(tokenizer.encode(sequence).ids)
tokenize_batch = lambda sequences: [tokenize(sequence) for sequence in sequences]

Data

In [4]:
train_sequences, train_fitnesses, test_sequences, test_fitnesses = get_GB1_dataset(
    tokenize=tokenize_batch, test_split=TEST_SPLIT
)

s:\Documents\master\ProGen\progen\progen2\.venv\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [11]:
n_data = 100
n_train = int(n_data * (1-TEST_SPLIT))
n_test = int(n_data * TEST_SPLIT)
train_sequences = train_sequences[0:n_train]
train_fitnesses = train_fitnesses[0:n_train]
test_sequences = test_sequences[0:n_test]
test_fitnesses = test_fitnesses[0:n_test]

Model

In [5]:
model = init_model(device=device)

In [6]:
for param in model.parameters():
    param.requires_grad = False

In [7]:
model.lm_head = torch.nn.Linear(in_features=1024, out_features=1, bias=True).to(device)

In [ ]:
print([param for param in model.parameters()])

Train

In [12]:
train(
    model=model,
    device=device,
    train_data=train_sequences,
    train_labels=train_fitnesses,
    test_data=test_sequences,
    test_labels=test_fitnesses,
    loss_function=torch.nn.functional.l1_loss,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    n_epochs=N_EPOCHS,
)

Training: mean test loss = 4.469400882720947


s:\Documents\master\code\llm\lib\model\train.py:39: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = loss_function(output, label)


Training: mean test loss = 0.60841304063797
